In [50]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [51]:
#import iris data
#from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
#iris = fetch_ucirepo(id=53) 
iris = pd.read_csv('Synthetic_IRIS_Dataset_Ray.csv', header=None, skiprows=6)
  
# data (as pandas dataframes) 
#X = iris.data.features 
#y = iris.data.targets 
X = iris.iloc[:, 0:4]
y = iris.iloc[:, 4:5]
print(X)
print(y)
  
# metadata 
#print(iris.metadata) 
  
# variable information 
#print(iris.variables) 

num_groups = 10

         0    1    2    3
0      6.3  2.7  1.4  0.3
1      6.5  2.7  6.1  1.9
2      5.9  3.6  1.1  0.3
3      5.4  2.6  1.1  1.4
4      6.5  3.2  1.4  1.8
...    ...  ...  ...  ...
49731  6.3  3.3  5.5  2.4
49732  5.7  3.1  3.9  1.9
49733  5.7  2.7  0.9  1.6
49734  4.8  3.0  3.3  1.4
49735  5.6  2.7  4.3  1.3

[49736 rows x 4 columns]
                   4
0          I. setosa
1       I. virginica
2          I. setosa
3      I. versicolor
4      I. versicolor
...              ...
49731   I. virginica
49732  I. versicolor
49733      I. setosa
49734      I. setosa
49735  I. versicolor

[49736 rows x 1 columns]


In [52]:
#create class for model creation
#from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

class CustomModel:
    def __init__(self, X=None, y=None):
        self.m = num_groups 
        subsize = int(X.shape[0] / self.m)
        self.model, self.X, self.y = [], [], []
        for i in range(self.m):
            self.model.append(MLPClassifier(hidden_layer_sizes=(15,10,5), max_iter=300,activation = 'relu',solver='adam',random_state=1))
            self.X.append(X[i*subsize:(i+1)*subsize].values)
            self.y.append(y[i*subsize:(i+1)*subsize].values)

    def dropData(self, groupid, cnt_Xy):
        #self.X[groupid] = self.X[groupid].drop(index=i)
        #self.y[groupid] = self.y[groupid].drop(index=i)
        #del self.X[groupid][1:cnt_Xy]
        #del self.y[groupid][1:cnt_Xy]
        self.X[groupid] = self.X[groupid][cnt_Xy+1:]
        self.y[groupid] = self.y[groupid][cnt_Xy+1:]

    def train(self, X_train=None, y_train=None):
        #if X_train is not None and y_train is not None:
        #    self.X = X_train
        #    self.y = y_train
        for i in range(self.m):
            self.model[i].fit(self.X[i], self.y[i])

    def train_one_group(self, groupid, X_train=None, y_train=None):
        #if X_train is not None and y_train is not None:
        #    self.X = X_train
        #    self.y = y_train
        self.model[groupid].fit(self.X[groupid], self.y[groupid])
        
    def evaluate(self, X_test, y_test):
        pdics = []
        for i in range(self.m): #self.m
            y_pred = self.model[i].predict(X_test)
            if i == 0:
                for pred in y_pred:
                    pdic = dict()
                    pdic[pred] = 1
                    pdics.append(pdic)
                
            else:
                for j in range(y_pred.shape[0]):
                    if y_pred[j] in pdics[j]:
                        pdics[j][y_pred[j]] += 1
                    else:
                        pdics[j][y_pred[j]] = 1
        for i in range(y_pred.shape[0]):
            mcount = pdics[i][y_pred[i]]
            for key, value in pdics[i].items():
                if value > mcount:
                    y_pred[i] = key
                    mcount = value
        accuracy = accuracy_score(y_test, y_pred)
        return accuracy

In [53]:
#create original model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=50)

original_model = CustomModel(X_train, y_train)
original_model.train()

# Evaluate the model with testing data
test_accuracy = original_model.evaluate(X_test, y_test)

print(f"Testing accuracy: {test_accuracy}")

Testing accuracy: 0.8685162846803377


In [54]:
#create unlearned model
unlearned_model = CustomModel(X_train, y_train)
cnt_drop_per_group = int(X.shape[0] / num_groups * 0.1) 

#test1. drop data in multiple groups
for i in range(10):
    unlearned_model.dropData( i, cnt_drop_per_group)

unlearned_model.train()

# Evaluate the model with testing data
test_accuracy_unlearned = unlearned_model.evaluate(X_test, y_test)

print(f"Testing accuracy: {test_accuracy_unlearned}")

Testing accuracy: 0.8677121029352634


In [55]:
#test2. drop the same amount of data only from group 1&2
unlearned_model.dropData( 1, cnt_drop_per_group*5)
unlearned_model.dropData( 2, cnt_drop_per_group*5)
unlearned_model.train_one_group(1)
unlearned_model.train_one_group(2)

# Evaluate the model with testing data
test_accuracy_unlearned = unlearned_model.evaluate(X_test, y_test)

print(f"Testing accuracy: {test_accuracy_unlearned}")

ValueError: Found array with 0 sample(s) (shape=(0, 4)) while a minimum of 1 is required by MLPClassifier.